# Python 2.7 编码问题

### 1.编译器对中文的识别

在使用 python.exe 运行.py文件时，提示

SyntaxError: Non-ASCII character '\xe4' in file test.py on line 1, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details

需要在脚本文件开头加入

In [2]:
# -*- coding: UTF-8 -*-

表明编译器需要以utf-8编码方式（而不是ascII）识别文件中的字符

### 2.直接键入的字符串的格式

对于英文文本中出现的所有字符，数字和标点，python 2.7将以上文本存储为str类型，编码方式为ascII(utf-8)

utf-8在英文部分编码与ascII一致

In [6]:
english = '123abc,.?'
type(english)

str

对于中文字符（符号），python2.7依然将文本存储为str类型，编码方式为utf-8

In [7]:
chinese = '中文'
type(chinese)

str

编译器对str类型的解码方式为ascII，每个中文字符会被拆解为三份（因为utf-8对不同字符采用不同长度编码）

In [9]:
[element for element in chinese]

['\xe4', '\xb8', '\xad', '\xe6', '\x96', '\x87']

In [12]:
chinese#这里的输出在python.exe 2.7.12中为正确的

'\xe4\xb8\xad\xe6\x96\x87'

### 3.Unicode编码与转码

为了让程序正确的切分字符串，需要将文本的编码格式改为unicode

In [14]:
# 当前格式为utf-8,'decode'为unicode
english = english.decode('utf-8')
type(english)

unicode

In [15]:
chinese = chinese.decode('utf-8')
type(chinese)

unicode

In [16]:
chinese2 = u'中文'
type(chinese2)

unicode

unicode 与 str 合并时默认转换为unicode

In [19]:
type(u'中文'+'english')

unicode

但由于转换的方法相当于 *.decode('ascII')，而汉字不能被ascII解码，所以报错

In [21]:
type(u'unicode中文'+'utf8中文')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe4 in position 4: ordinal not in range(128)

大部分包含汉字的文档采用utf-8编码，因此在读入文档后需要decode为unicode进行处理

In [30]:
with open('test.txt','r') as fhand:
    this_line = fhand.readline()
this_line = this_line.decode('utf-8')
print(this_line)
print(type(this_line))

标题title

<type 'unicode'>


在写入时encode为utf-8进行存储

In [31]:
with open('test.txt','w') as fhand:
    fhand.write(this_line.encode('utf-8'))

如果需要默认将所有脚本中的字符串编码为unicode(相当于对全部的 '字符串'->u'字符串')，可以在脚本开头

In [32]:
from __future__ import unicode_literals

这时

In [33]:
type('中文')

unicode

这样修改的问题是，在python 2.7中默认输入的字符串是str，所以默认解码相当于 *.decode('ascII')

在修改后，会产生很多unicode编码的纯英文文本

在作为某些（需要以str为参数）函数的输入时，被默认以ascII编码会产生错误

In [41]:
'utf8中文'.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode characters in position 4-5: ordinal not in range(128)

或者隐式的

In [43]:
from datetime import datetime
now = datetime.now()
print now.strftime('%m月%d日 %H:%M')

UnicodeEncodeError: 'ascii' codec can't encode character u'\u6708' in position 2: ordinal not in range(128)

需要修改默认解码方式来调整encode,decode的默认类型

In [45]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

这样，解码

In [46]:
type(u'unicode中文'+b'utf8中文')

unicode

和编码中文都不会出错

In [53]:
now = datetime.now()
now.strftime('%m月%d日 %H:%M')

'06\xe6\x9c\x8828\xe6\x97\xa5 16:59'